<a href="https://colab.research.google.com/github/bearbearyu1223/llm-fine-tuning-playground/blob/main/finetune_falcon_7b_conversation_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine-tune [Falcon-7b-sharded model](https://huggingface.co/vilsonrodrigues/falcon-7b-sharded)** on [samsum](https://huggingface.co/datasets/samsum) can be found on Hugging Face.
Links to both the model and dataset are in the notebook.


##Installs and imports

In [1]:
#all installs
!pip install -q -U trl accelerate git+https://github.com/huggingface/peft.git
!pip install transformers==4.34.0
!pip install huggingface_hub==0.18.0
!pip install -q datasets bitsandbytes einops wandb


#all imports
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training, TaskType
from transformers import TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-h

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


##Notebook connection to Hugging face

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
notebook_login()

##Loading the dataset from hugging face and Formatting the Training Dataset

---



In [4]:
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.0 MB/s eta 0:00:00


In [5]:
dataset_name = "samsum"
dataset = load_dataset(dataset_name)

train_dataset = dataset['train']
eval_dataset = dataset['validation']
test_dataset = dataset['test']
dataset

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

##Loading the model and Setting up bitsandbytes config

We will use sharded version of falcon-7b model.


In [6]:
model_name = "vilsonrodrigues/falcon-7b-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00013-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/828M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

##Loading the tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

##Setting up the LoRA config

In [8]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 #16
lora_dropout = 0.1 #0.1
lora_rank = 16 #64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

##Load the trainer

In [9]:
output_dir = "falcon_7b_LoRA_dialogue_summarization"
per_device_train_batch_size = 16 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "cosine" #"constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    report_to="wandb"
)

##Passing arguments to the SFTT trainer

In [10]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['id'])):
        prompt= f"""
         ### Summarize the conversation below

         ### Dialogue:
         {example['dialogue'][i]}

         ### Summary:
         {example['summary'][i]}
        """
        output_texts.append(prompt)
    return output_texts

In [11]:
max_seq_length = 256

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,
    formatting_func=formatting_prompts_func,
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    args=training_arguments,
)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

In [12]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

##Train the model

You can check your training time if you are doing multiple experiments

In [13]:
import os
import time
import wandb
os.environ["WANDB_SILENT"] = "true"
wandb.init(project="falcon-7b-peft-lora-dialogue-summarization")

<IPython.core.display.Javascript object>

In [14]:
peft_model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.312200
20,2.024700
30,1.793900
40,1.665400
50,1.494400
60,2.132700
70,1.821500
80,1.702000
90,1.614000
100,1.460300


TrainOutput(global_step=1000, training_loss=1.6194404468536376, metrics={'train_runtime': 5392.4527, 'train_samples_per_second': 11.868, 'train_steps_per_second': 0.185, 'total_flos': 4.4715736602890035e+17, 'train_loss': 1.6194404468536376, 'epoch': 4.34})

##Save the model

In [15]:
#trainer.save() #if you want to save your model locally

##Push to hub

In [24]:
from huggingface_hub import create_repo
create_repo(output_dir)

RepoUrl('https://huggingface.co/bearbearyu1223/falcon_7b_LoRA_dialogue_summarization', endpoint='https://huggingface.co', repo_type='model', repo_id='bearbearyu1223/falcon_7b_LoRA_dialogue_summarization')

In [26]:
trainer.save_model(output_dir)
trainer.push_to_hub()

'https://huggingface.co/bearbearyu1223/falcon_7b_LoRA_dialogue_summarization/tree/main/'

##Inference

In [27]:
# Loading PEFT model
PEFT_MODEL = "bearbearyu1223/"+ output_dir
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/131M [00:00<?, ?B/s]

In [28]:
# Generate responses from both orignal model and fine-tuned model
def get_response(dialogue, max_new_tokens=10):
  prompt= f"""
         ### Summarize the conversation below

         ### Dialogue:
         {dialogue}

         ### Summary:

        """
  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to(torch.device("cuda:0"))
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=max_new_tokens, pad_token_id = peft_tokenizer.eos_token_id,
                                                                                                         eos_token_id = peft_tokenizer.eos_token_id,
                                                                                                         attention_mask = peft_encoding.attention_mask,
                                                                                                         temperature=0.1, top_p=0.5,  repetition_penalty=10.0, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)
  return peft_text_output

In [39]:
import re

def remove_incomplete_sentences(text):

  # Compile the regular expression for complete sentences.
  incomplete_sentence_regex = re.compile(r"(^.*[\.\?!]|^\S[^.\?!]*)")

  # Find all of the incomplete sentences in the text.
  complete_sentences = incomplete_sentence_regex.findall(text)
  text = " ".join(complete_sentences)

  # Return the text with the incomplete sentences removed.
  return text

text = "This is a sentence. This is not"
remove_incomplete_sentences(text)

'This is a sentence.'

In [51]:
import transformers

test_index=818
dialogue=test_dataset[test_index]['dialogue']
summary=test_dataset[test_index]['summary']
condense_rate=0.5
max_new_tokens=max(int(len(dialogue.strip().split())*condense_rate), len(summary.split()))

peft_output=get_response(dialogue, max_new_tokens)
sub = "### Summary:"
peft_summary = peft_output.split(sub)[1]
post_processed_peft_model_summary = remove_incomplete_sentences(peft_summary.strip())

pipeline = transformers.pipeline(
    "text-generation",
    model=peft_base_model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
sequences = pipeline(
    formatting_prompts_func(test_dataset)[test_index],
    max_new_tokens=max_new_tokens,
    temperature=0.1, top_p=0.5,  repetition_penalty=10.0, num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
post_processed_base_model_summary= remove_incomplete_sentences(peft_summary.strip())
dash_line = '-'.join('' for x in range(100))
print(dash_line)
print('BASELINE HUMAN SUMMARY:')
print(summary)
print(dash_line)
print('BASE MODEL SUMMARY:')
print(sequences[0]['generated_text'])
print(dash_line)
print('PEFT MODEL SUMMARY')
print(post_processed_peft_model_summary)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Rachel sends a list of Top 50 films of 2018. Janice watched almost half of them, Deadpool 2 and Avengers included.
---------------------------------------------------------------------------------------------------
BASE MODEL SUMMARY:

         ### Summarize the conversation below

         ### Dialogue:
         Rachel: <file_other>
Rachel: Top 50 Best Films of 2018
Rachel: :)
Janice: Omg, I've watched almost all 50... xDD
Spencer: Hahah, Deadpool 2 also??
Janice: Yep
Spencer: Really??
Janice: My bf forced me to watch it xD
Rachel: Hahah
Janice: It wasn't that bad
Janice: I thought it'd be worse
Rachel: And Avengers? :D
Janice: 2 times
Rachel: Omg
Janice: xP
Rachel: You are the best gf in the world
Rachel: Your bf should appreciate that ;-)
Janice: He does
Janice: x)

         ### Summary:
         Rachel sends a list of Top 50 films of 2018. Janice watched almos

### Evaluation

In [35]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install evaluate
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f5bc57e8dcd97ba08ba22cc5b0d7f3b8796c381b29a529d91dc1c15807c4714f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [40]:
from tqdm import tqdm

references = []
predictions = []

for sample in tqdm(test_dataset):
  summary=sample['summary']
  dialogue=sample['dialogue']
  max_new_tokens=int(len(dialogue.strip().split())*condense_rate)
  peft_output=get_response(dialogue, max_new_tokens)
  sub = "### Summary:"
  peft_summary = peft_output.split(sub)[1]
  post_processed_peft_model_summary = remove_incomplete_sentences(peft_summary.strip())
  references.append(summary)
  predictions.append(post_processed_peft_model_summary)

100%|██████████| 819/819 [56:08<00:00,  4.11s/it]


In [41]:
import evaluate

metric = evaluate.load('rouge')
results = metric.compute(predictions=predictions, references=references)

print(f"Rogue1: {results['rouge1']* 100:2f}%")
print(f"rouge2: {results['rouge2']* 100:2f}%")
print(f"rougeL: {results['rougeL']* 100:2f}%")
print(f"rougeLsum: {results['rougeLsum']* 100:2f}%")

Rogue1: 27.290604%
rouge2: 6.762303%
rougeL: 21.281129%
rougeLsum: 21.276660%
